In [19]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd
import evaluate

In [20]:
dataset = load_dataset("gretelai/synthetic_text_to_sql")

print("Available data subsets:", dataset.keys())

df_train = pd.DataFrame(dataset['train'])

print("First cell for each column for 1st row")
print(df_train.head(n=1))

Available data subsets: dict_keys(['train', 'test'])
First cell for each column for 1st row
     id    domain                                 domain_description  \
0  5097  forestry  Comprehensive data on sustainable forest manag...   

  sql_complexity                   sql_complexity_description  \
0    single join  only one join (specify inner, outer, cross)   

             sql_task_type                          sql_task_type_description  \
0  analytics and reporting  generating reports, dashboards, and analytical...   

                                          sql_prompt  \
0  What is the total volume of timber sold by eac...   

                                         sql_context  \
0  CREATE TABLE salesperson (salesperson_id INT, ...   

                                                 sql  \
0  SELECT salesperson_id, name, SUM(volume) as to...   

                                     sql_explanation  
0  Joins timber_sales and salesperson tables, gro...  


In [21]:
def check_missing_data(dataset, columns):
    for split in dataset.keys():
        print(f"Checking '{split}' split...")
        for column in columns:
            if column not in dataset[split].column_names:
                print(f"  Column '{column}' not found in the dataset!")
                continue
            missing_count = sum(1 for example in dataset[split] if not example[column] or example[column].strip() == "")
            print(f"  {column}: {missing_count} missing or empty entries")
        print()

print("Dataset columns:", dataset["train"].column_names)
check_missing_data(dataset, columns=["sql_prompt", "sql"])

Dataset columns: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation']
Checking 'train' split...
  sql_prompt: 0 missing or empty entries
  sql: 0 missing or empty entries

Checking 'test' split...
  sql_prompt: 0 missing or empty entries
  sql: 0 missing or empty entries



In [22]:
def normalize_text(text):
    text = text.lower().strip().replace("\n", " ")
    return text

tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [23]:
def preprocess(example):
    input_text = "Translate to SQL: " + normalize_text(example["sql_prompt"])
    output_text = normalize_text(example["sql"])

    model_inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(output_text, truncation=True, padding="max_length", max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_ds = dataset.map(preprocess, remove_columns=dataset["train"].column_names)

available_splits = dataset.keys()
train_ds = tokenized_ds["train"]
validation_ds = tokenized_ds["test"] if "test" in available_splits else None
test_ds = tokenized_ds["test"] if "test" in available_splits else None

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5851 [00:00<?, ? examples/s]

In [24]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

training_args = Seq2SeqTrainingArguments(
    output_dir="./sql_model",
    eval_strategy="epoch",  # Changed from evaluation_strategy
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=validation_ds,
    tokenizer=tokenizer
)

trainer.train()

C:\Users\szymo\AppData\Local\Temp\ipykernel_16792\2150548209.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
C:\Users\szymo\Desktop\ProgramingStuff\VisualStudioCodeProjects\llm-fine-tuning\venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
def evaluate_model(test_dataset):
    metric = evaluate.load("sacrebleu")

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_labels = [[label] for label in decoded_labels]
        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        return {"bleu": result["score"]}

    results = trainer.evaluate(eval_dataset=test_dataset, compute_metrics=compute_metrics)
    return results

if test_ds:
    test_results = evaluate_model(test_ds)
    print(test_results)

In [ ]:
def driver(question):
    inputs = tokenizer("Translate to SQL: " + question, return_tensors="pt").input_ids
    outputs = model.generate(inputs, max_length=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(driver("Find all customers who ordered in 2023"))